# Fine-tuning модели Stable Diffusion с помощью метода DreamBooth

## Setup & Imports

In [ ]:
import numpy as np
import pandas as pd
import torch
import clip
import textwrap
from IPython.display import display
from PIL import Image, ImageDraw, ImageFont
from torchmetrics.functional.pairwise import pairwise_cosine_similarity

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
train_images = [Image.open(f"data/images/train/0{i}.jpg") for i in range(1, 10)]
test_images = [Image.open(f"data/images/test/test_0{i}.jpg") for i in range(1, 9)]

## Basic utils

In [ ]:
def image_grid(imgs, num_rows, num_cols, row_names=None, col_names=None):
    assert ((row_names is None or len(row_names) == num_rows) and
            (col_names is None or len(col_names) == num_cols))

    w, h = imgs[0].size
    rows_bias = int(row_names is not None)
    cols_bias = int(col_names is not None)

    def text_image(text):
        image = Image.new('RGB', (w, h), (255, 255, 255))
        draw = ImageDraw.Draw(image)
        font = ImageFont.truetype("arial.ttf", w // 10)
        lines = textwrap.wrap(text, width=20)
        y_text = 0
        for line in lines:
            bbox = font.getbbox(line)
            line_width, line_height = bbox[2] - bbox[0], bbox[1] - bbox[3]
            draw.text(((w - line_width) / 2, y_text), line, font=font, fill=(0, 0, 0))
            y_text -= line_height
        return image
    
    grid = Image.new("RGB", size=((num_cols + rows_bias) * w, (num_rows + cols_bias) * h))

    if col_names is not None:
        for i in range(rows_bias, num_cols + rows_bias):
            grid.paste(text_image(col_names[i - rows_bias]), box=(i * w, 0))

    if row_names is not None:
        for i in range(cols_bias, num_rows + cols_bias):
            grid.paste(text_image(row_names[i - cols_bias]), box=(0, i * h))

    for i, img in enumerate(imgs):
        grid.paste(img, box=((i % num_cols + rows_bias) * w, (i // num_cols + cols_bias) * h))
        
    return grid

## Данные

### Train images

Для fine-tuning'а модели с помощью DreamBooth был собран небольшой датасет с персонажем Момо из мультсериала "Аватар: Легенда об Аанге".

Для дообучения модели было собрано 9 изображений. В дальнейшем в качестве экспериментов для дообучения модели использовалось различное количество изображений, поэтому собранные изображения были отсортированы в соотвествии с предполагаемой пользой для модели (чтобы использовать top-N изображений для обучения). Тренировочные изображения представлены ниже:

In [ ]:
image_grid(train_images, num_rows=3, num_cols=3)

### Test images

Также было собрано 8 изображений с Момо в разнообразных сценах из мультфильма. На данных изображениях присутствуют другие герои и предметы, контекст изображений наиболее разнообразный. Данные изображения будут использоваться для оценки дообученной модели.

In [ ]:
image_grid(test_images, num_rows=3, num_cols=3)

## Оценка модели

Для изображений из тестовой выборки были написаны примерные описания этих изображений. Эти описания будут использоваться в качестве промптов для генерации изображений. Далее сгенерированные изображения будут сраваниваться с тестовыми изображениями по метрике CLIP-I, предложенной в статье. Метрика CLIP-I представляет собой усреднённое попарное косинусное расстояние между эмбеддингами сгенерированных и реальных изображений. Также для оценки моделей будет использоваться предложенная в статье метрика CLIP-T, которая представляет собой усреднённое попарное косинусное расстояние между эмбеддингами сгенерированных изоюражений и промптами к ним. Для этого были написаны дополнительные промпты для генерации, многие из которых были взяты из статьи.

In [ ]:
from data.prompts import prompts

Первые 8 промптов соответствуют описаниям тестовых изображений.

In [ ]:
unique_token, class_token = "sks", "lemur"
prompts = [p.format(unique_token, class_token) for p in prompts]
prompts


Итого, для оценки моделей будут использоваться следующие аспекты:
- Test CLIP-I: изображения из тестовой выборки попарно сравниваются со сгенерированными по описаниям изображениями, затем значения метрики CLIP-I для этих изображений усредняются (то есть получается 8 пар изображений - 8 чисел, которые усредняются);
- Overall CLIP-I: Для всех реальных изображений, которые были собраны (как train, так и test), и для всех сгенерированных по заготовленным промптам изображений будет считать метрика CLIP-I (то есть попарно сравниваются все изображения);
- CLIP-T: подсчёт метрики для всех промптов и сгенерированных по ним изображений.
- Визуальная оценка изображений

In [ ]:
class Metrics:
    def __init__(self, device):
        self.device = device
        self.model, self.preprocess = clip.load("ViT-B/32", device=device)
        
    def _get_image_embedding(self, image):
        image = self.preprocess(image).unsqueeze(0).to(self.device)
        image_embedding = self.model.encode_image(image)

        return image_embedding.squeeze()

    def _get_images_embeddings(self, images):
        images_embeddings = [self._get_image_embedding(image) for image in images]

        return torch.stack(images_embeddings)

    def _get_prompt_embedding(self, prompt):
        prompt = clip.tokenize(prompt).to(self.device)
        prompt_embedding = self.model.encode_text(prompt)

        return prompt_embedding.squeeze()

    def _get_prompts_embeddings(self, prompts):
        prompts_embeddings = [self._get_prompt_embedding(prompt) for prompt in prompts]

        return torch.stack(prompts_embeddings)

    def CLIP_I(self, generated_images, real_images):
        generated_images_embeddings = self._get_images_embeddings(generated_images)
        real_images_embeddings = self._get_images_embeddings(real_images)

        return pairwise_cosine_similarity(generated_images_embeddings, real_images_embeddings).mean().detach().cpu().item()
    
    def CLIP_T(self, generated_images, prompts):
        generated_images_embeddings = self._get_images_embeddings(generated_images)
        prompts_embeddings = self._get_prompts_embeddings(prompts)

        return pairwise_cosine_similarity(generated_images_embeddings, prompts_embeddings).mean().detach().cpu().item()

In [ ]:
class EvaluationReport:
    def __init__(self, test_images, prompts):
        self.test_images = test_images
        self.prompts = prompts
        self.images = []
        self.metrics = Metrics(device)
    
    def _get_experiment_metrics(self, images):
        test_clip_i = np.mean(
            [
                self.metrics.CLIP_I([gen_img], [test_img])
                for gen_img, test_img in zip(images[: len(self.test_images)], self.test_images)
            ]
        )
        overall_clip_i = self.metrics.CLIP_I(images, self.test_images)
        clip_t = self.metrics.CLIP_T(images, self.prompts)

        return tuple(map(lambda x: round(x, 3), (test_clip_i, overall_clip_i, clip_t)))

    def _show_metrics(self):
        metrics_df = pd.DataFrame(index=["Test CLIP-I", "Overall CLIP-I", "CLIP-T"])
        for images, col_name in zip(self.images_sets, self.col_names):
            metrics_df[col_name] = self._get_experiment_metrics(images)
        
        display(metrics_df)

    def _show_images(self):
        grid_images = []
        for i in range(len(self.prompts)):
            grid_images.extend([images[i] for images in self.images_sets])
        
        display(image_grid(grid_images, num_rows=len(self.prompts), num_cols=len(self.col_names),
                                row_names=self.prompts, col_names=self.col_names))

    def make_report(self, images_paths: list[str], col_names: list[str], show_images=True):
        assert len(images_paths) == len(col_names)
        self.images_sets = [[Image.open(f"{images_path}/{i}.jpg") for i in range(len(self.prompts))] for images_path in images_paths]
        self.col_names = col_names

        self._show_metrics()
        if show_images:
            self._show_images()

## Эксперименты

In [ ]:
eval_report = EvaluationReport(test_images, prompts)

> Иногда при генерации получались чёрные изображения из-за safety_checker'a

### Количество изображений для обучения

В статье утверждается, что для качественного fine-tuning'a достаточно 3-5 изображений. Для оценки влияния количества изображений в тренировочной выборке были проведены эксперименты со стандартными гиперпараметрами, которые задаются в исходном репозитории скрипта. Были проведены эксперименты с 3, 6, 9 изображениями.

In [ ]:
eval_report.make_report(
    images_paths=[
        "checkpoints/output-images_3-without_pp-lr_0.0001-numsteps_500-rank_16/images",
        "checkpoints/output-images_6-without_pp-lr_0.0001-numsteps_500-rank_16/images",
        "checkpoints/output-images_9-without_pp-lr_0.0001-numsteps_500-rank_16/images"
    ],
    col_names=["3 images", "6 images", "9 images"]
)

На последних 3 картинках из тренировочной выборки (то есть на тех, которые участвовали только в обучении модели на 9 изображениях) персонаж изображен в основном крупным планом - и это прослеживается в сгенерированных изображениях: для эксперимента с 9 изображениями модель как будто уделяет больше внимания лицу (глаза заметно больше, лицо более округлое, в целом лицо немного другое по сравнению с другими моделями), и меньше внимания остальному телу, вследствие чего искажаются некоторые его детали. То есть модель немного переобучилась на Момо крупным планом. Это же прослеживается и в метриках, которые заметно ниже метрик для других экспериментов (также здесь можно увидеть, что метрики CLIP-I относительно других экспериментов чуть похуже, чем CLIP-T, то есть и хотя изображения лучше (в относительной мере) соответствуют промптам, всё же они меньше похожи на реальные изображения с Момо).

Что касается моделей, обученных на 3 и 6 изображениях: хоть по метрикам себя немного лучше показывает модель на 3 изображениях, визуально кажется, что модель на 6 изображениях уделяет больше внимания деталям и генерирует более реалистичные изображения. Гиперпараметры модели могут быть подобраны неидеально, и различие в метриках в данном случае не обязательно может говорить о превосходстве одной из моделей. Я думаю, большее количество изображений для обучения может помочь модели лучше запомнить детали и сделать генерацию чуть разнообразней в дальнейшем при подборе гиперпараметров, поэтому было решено в дальнейших экспериментах использовать 6 изображений для обучения.

### Использование Prior Preservation Loss

In [ ]:
eval_report.make_report(
    images_paths=[
        "checkpoints/output-images_6-without_pp-lr_0.0001-numsteps_500-rank_16/images",
        "checkpoints/output-images_6-with_pp-loss_weight_0.0001-lr_0.0001-numsteps_500-rank_16/images",
        "checkpoints/output-images_6-with_pp-loss_weight_0.001-lr_0.0001-numsteps_500-rank_16/images",
        "checkpoints/output-images_6-with_pp-loss_weight_0.01-lr_0.0001-numsteps_500-rank_16/images",
        "checkpoints/output-images_6-with_pp-loss_weight_0.1-lr_0.0001-numsteps_500-rank_16/images",
    ],
    col_names=["W/o PPL", "W/ PPL, weight = 0.0001", "W/ PPL, weight = 0.001",
               "W/ PPL, weight = 0.01", "W/ PPL, weight = 0.1"]
)

Здесь видно, что с увеличением веса PPL изображения становятся чуть более реалистичными (менее мультяшными), а также особенно становится заметным преобладание класса lemur - в последней колонке с наибольшим весом loss'а на многих изображения вообще теряются свойства персонажа. Эту же тенденцию можно проследить и в получившихся метриках: так как для CLIP-I в качестве реальных изображений используются изображения из мультсериала, то значения метрик закономерно снижаются при увеличении веса loss'а, так как сгенерированные изображения сравниваются с более реалистичными (с более похожием на изображения из реального мира). В то же время метрика CLIP-T с увеличением веса loss'а растет, так как изображения становятся более похожими на промпты, в которых участвует название класса и детали описания сцены.

Здесь также следует учесть то, что при использовании PPL выбранные learning rate и число шагов могут быть неоптимальными - при их увеличении детали персонажа всё же могут сохраняться. Но эксперименты с использованием PPL не показали хороших результатов, Поэтому было решено проводить дальнейшие эксперименты без использования PPL.

### Lora Rank

In [ ]:
eval_report.make_report(
    images_paths=[
        "checkpoints/output-images_6-without_pp-lr_0.0001-numsteps_500-rank_2/images",
        "checkpoints/output-images_6-without_pp-lr_0.0001-numsteps_500-rank_4/images",
        "checkpoints/output-images_6-without_pp-lr_0.0001-numsteps_500-rank_8/images",
        "checkpoints/output-images_6-without_pp-lr_0.0001-numsteps_500-rank_16/images",
        "checkpoints/output-images_6-without_pp-lr_0.0001-numsteps_500-rank_32/images",
    ],
    col_names=["Rank = 2", "Rank = 4", "Rank = 8", "Rank = 16", "Rank = 32"]
)

Визуально кажется, что примеры, сгенерированные моделью с Rank = 16, лучше изображают персонажа и содержат меньше различных артефактов, связанных с его деталями, о чём свидетельствуют неплохие значения метрики CLIP-I. Также у этой модели неплохие значения метрики CLIP-T, хотя и видно, что на других изображениях контекст сохраняется лучше и выглядит более реалистично. Будем в дальнейших экспериментах использовать Rank, равный 16.

### Learning Rate & Number of steps

Основной проблемой при дообучении модели с помощью DreamBooth может являться переобучение. Поэтому важными параметрами являются Learning Rate и число шагов для обучения. Для получения качественных изображений нужно найти оптимальное соотношение между ними, поэтому в следующих экспериментах для разных learning rate пробовалиось раличное значение количества шагов.

##### Learning Rate = 1e-5

In [ ]:
eval_report.make_report(
    images_paths=[
        "checkpoints/output-images_6-without_pp-lr_1e-05-numsteps_500-rank_16/images",
        "checkpoints/output-images_6-without_pp-lr_1e-05-numsteps_800-rank_16/images",
        "checkpoints/output-images_6-without_pp-lr_1e-05-numsteps_1000-rank_16/images",
        "checkpoints/output-images_6-without_pp-lr_1e-05-numsteps_1200-rank_16/images",
    ],
    col_names=["Steps = 500", "Steps = 800", "Steps = 1000", "Steps = 1200"]
)

##### Learning Rate = 1e-4

In [ ]:
eval_report.make_report(
    images_paths=[
        "checkpoints/output-images_6-without_pp-lr_0.0001-numsteps_500-rank_16/images",
        "checkpoints/output-images_6-without_pp-lr_0.0001-numsteps_800-rank_16/images",
        "checkpoints/output-images_6-without_pp-lr_0.0001-numsteps_1000-rank_16/images",
        "checkpoints/output-images_6-without_pp-lr_0.0001-numsteps_1200-rank_16/images",
    ],
    col_names=["Steps = 500", "Steps = 800", "Steps = 1000", "Steps = 1200"]
)

##### Learning Rate = 1e-3

In [ ]:
eval_report.make_report(
    images_paths=[
        "checkpoints/output-images_6-without_pp-lr_0.001-numsteps_500-rank_16/images",
        "checkpoints/output-images_6-without_pp-lr_0.001-numsteps_800-rank_16/images",
        "checkpoints/output-images_6-without_pp-lr_0.001-numsteps_1000-rank_16/images",
        "checkpoints/output-images_6-without_pp-lr_0.001-numsteps_1200-rank_16/images",
    ],
    col_names=["Steps = 500", "Steps = 800", "Steps = 1000", "Steps = 1200"]
)

Видно, что слишком маленький learning rate приводит к тому, что модель не выучивает представление о персонаже. При большем learning rate видно, что сам персонаж прорисовывается лучше, но совсем плохо генерируется контекст, модель сильно теряет в разнообразии генерации. При этом увеличение количества шагов помогает исправить ситуацию и улучшить фон изображеня.

По итогу проведенных экспериментов во многом плохо получилось сгенерировать хороший и качественный контекст. Промпты, которые были составлены для тестовых изображений, оказались слишком сложными для данной задачи, им почти никогда не удавалось соответствовать. Также и для других, более простых, промтов часто оказывалось плохое качество генерации. Ещё одной сложностью был домен персонажа: так как сам персонаж несуществующий и сделан в анимационном оформлении, то класс "Лемур", хоть и является наболее близким (в мультике Момо - это крылатый лемур), но всё же далёк от него, что сказалось на качестве генерации изображений и оценке с помощью предложенных метрик. Тем менее зачастую получились хорошие изображения с персонажем, во многом с сохранением его особенностей.